In [16]:
from tensorflow.keras.models import Model as KerasModel
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Embedding, LSTM
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import tensorflow as tf

In [17]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.3.1
Keras Version: 2.4.0

Python 3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 15:37:01) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.3
Scikit-Learn 0.24.1
GPU is available


In [18]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


In [35]:
train_dir = 'dataset/train_images'
val_dir = 'dataset/val_images'
test_dir = 'dataset/test_images'
train_dir2 = 'train'

#datagen = ImageDataGenerator(1./255)
train_generator = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir2,
    validation_split=0.2,
    batch_size=32,
    image_size=(256,256),
    seed=123,
    color_mode='rgb',
    subset = 'training'
)
val_generator = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir2,
    validation_split=0.2,
    batch_size=32,
    image_size=(256,256),
    seed=123,
    color_mode='rgb',
    subset = 'validation'
)

dataGenerator_test = ImageDataGenerator(rescale=1./255)
generator_test = dataGenerator_test.flow_from_directory(
       'dataset/test_images',
        target_size=(256, 256),
        batch_size=1,
        class_mode='binary')
print('-------------test-------------')
print(generator_test.class_indices)
print('-------------train&validation-------------')
print(train_generator.class_names)

Found 8104 files belonging to 2 classes.
Using 6484 files for training.
Found 8104 files belonging to 2 classes.
Using 1620 files for validation.
Found 4 images belonging to 2 classes.
-------------test-------------
{'fake': 0, 'real': 1}
-------------train&validation-------------
['fake', 'real']


In [20]:
for batch, label in train_generator:
    print(batch.shape)
    print(label.shape)
    break

(32, 256, 256, 3)
(32, 2)


In [37]:
#CNN model
model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
    Conv2D(64,(3,3), padding='same',activation='relu'),
    Conv2D(64,(3,3), padding='same',activation='relu'),
    MaxPooling2D((2,2),strides=(2,2)),
    Conv2D(128,(3,3), padding='same',activation='relu'),
    Conv2D(128,(3,3), padding='same',activation='relu'),
    MaxPooling2D((2,2),strides=(2,2)),
    Conv2D(256,(3,3), padding='same',activation='relu'),
    Conv2D(256,(3,3), padding='same',activation='relu'),
    Conv2D(256,(3,3), padding='same',activation='relu'),
    Conv2D(256,(3,3), padding='same',activation='relu'),
    MaxPooling2D((2,2),strides=(2,2)),
    Flatten(),
    Dense(units=512,activation="relu"),
    Dense(units=2,activation='softmax')
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_3 (Rescaling)      (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 64, 64, 128)      

In [38]:
optimizer = SGD(lr=0.01)
model.compile(optimizer='adam',
              loss = tf.keras.losses.categorical_crossentropy, 
              metrics=['accuracy'])

In [39]:
print(len(train_generator))
print(len(val_generator))

203
51


In [40]:
history = model.fit(train_generator,
                    steps_per_epoch = len(train_generator),
                    validation_data=val_generator,
                    validation_steps=len(val_generator),
                    epochs=10)

Epoch 1/10


ValueError: in user code:

    C:\Users\xiaob\anaconda3\envs\deepfake\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\xiaob\anaconda3\envs\deepfake\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\xiaob\anaconda3\envs\deepfake\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\xiaob\anaconda3\envs\deepfake\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\xiaob\anaconda3\envs\deepfake\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\xiaob\anaconda3\envs\deep

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(20)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()